In [3]:

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean

# 初始化 Spark 会话
spark = SparkSession.builder.appName('iteration3').getOrCreate()


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/zhoudaozhen/anaconda3/lib/python3.11/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [2]:
#!/Users/zhoudaozhen/anaconda3/bin/python -m pip install findspark
#!/Users/zhoudaozhen/anaconda3/bin/python -m pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.8 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 5.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=5172d9b0f3a9c3d1426407eb0f52526d30ae30cdb78a1c5d47495b3d16835d09
  Stored in directory: /Users/zhoudaozhen/Library/Caches/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
import os
import numpy as np
import warnings
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

warnings.filterwarnings('ignore', category=FutureWarning)


In [5]:
# 分别为每个年份创建DataFrame
data_2013 = spark.read.csv("2013.csv")
data_2015 = spark.read.csv("2015.csv")
data_2016 = spark.read.csv("2016.csv")
data_2017 = spark.read.csv("2017.csv")
data_2018 = spark.read.csv("2018.csv")
data_2019 = spark.read.csv("2019.csv")
data_2020 = spark.read.csv("2020.csv")
data_2021 = spark.read.csv("2021.csv")
data_2022 = spark.read.csv("2022.csv")
data_2023 = spark.read.csv("2023.csv")




NameError: name 'spark' is not defined

In [ ]:



folder_path = '/Users/zhoudaozhen/Desktop/722/iteration3'

data_frames = []
common_columns = None

for year in range(2013, 2024):
    if year != 2014:
        file_path = os.path.join(folder_path, f"{year}.csv")
        if os.path.exists(file_path):
            df = spark.read.csv(file_path)
            data_frames.append(df)
            if common_columns is None:
                common_columns = set(df.columns)
            else:
                common_columns.intersection_update(df.columns)

filtered_data_frames = [df[list(common_columns)] for df in data_frames]

combined_data = pd.concat(filtered_data_frames, ignore_index=True)


data_attribute = pd.read_excel('P_Data_Extract_From_World_Development_Indicators.xlsx')



In [ ]:
combined_data

In [ ]:
data_2013.head(15)

In [ ]:
round(data_2013.describe().transpose(),2)

In [ ]:
# 选择数值型数据列
numerical_cols = data_2013.select_dtypes(include=['number']).columns
numerical_cols = [col for col in numerical_cols if col not in ['Year', 'Happiness (Yesterday)']]
# 设置画布大小
plt.figure(figsize=(15, len(numerical_cols) // 3 * 5))

# 为每个数值型属性画图
for i, col in enumerate(numerical_cols):
    plt.subplot(len(numerical_cols) // 3 + 1, 3, i + 1)  # 确保有足够的行数容纳所有图
    sns.histplot(data_2013[col], kde=True)

# 调整子图间距
plt.tight_layout()
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/pic/9", format='png')
plt.show()


In [ ]:
data_2020.head(10)

In [ ]:
round(data_2020.describe().transpose(),2)

In [ ]:
# 选择数值型数据列
numerical_cols = data_2020.select_dtypes(include=['number']).columns
numerical_cols = [col for col in numerical_cols if col not in ['Year', 'Happiness (Yesterday)','Happiness Rank']]
# 设置画布大小
plt.figure(figsize=(15, len(numerical_cols) // 3 * 5))

# 为每个数值型属性画图
for i, col in enumerate(numerical_cols):
    plt.subplot(len(numerical_cols) // 3 + 1, 3, i + 1)  # 确保有足够的行数容纳所有图
    sns.histplot(data_2020[col], kde=True)

# 调整子图间距
plt.tight_layout()
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/pic/12", format='png')
plt.show()


In [ ]:
print(f"number of rows: {combined_data.shape[0]}, number of colnums: {combined_data.shape[1]}")

In [ ]:
combined_data.info()

In [ ]:
round(combined_data.describe().transpose(),2)

In [ ]:
print(f"number of rows: {data_attribute.shape[0]}, number of colnums: {data_attribute.shape[1]}")

In [ ]:
data_attribute.info()

In [ ]:
data_attribute.replace('..', np.nan, inplace=True)

In [ ]:
data_attribute.info()

In [ ]:
round(data_attribute.describe().transpose(),2)

In [ ]:
data_attribute

In [ ]:
# 选择数值型数据列
numerical_cols = data_attribute.select_dtypes(include=['number']).columns
numerical_cols = [col for col in numerical_cols if col not in ['Year', 'Happiness (Yesterday)','Happiness Rank']]
# 设置画布大小
plt.figure(figsize=(15, len(numerical_cols) // 5 * 5))

# 为每个数值型属性画图
for i, col in enumerate(numerical_cols):
    plt.subplot(len(numerical_cols) // 5 + 1, 5, i + 1)  # 确保有足够的行数容纳所有图
    sns.histplot(data_attribute[col], kde=True)

# 调整子图间距
plt.tight_layout()
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/pic/17.png", format='png')
plt.show()


In [ ]:
# 假设data_2015是你的DataFrame，并且它已经加载
regions = data_2015['Region'].unique()

# 每行显示三张图
num_cols = 3
num_rows = int(np.ceil(len(regions) / num_cols))

fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols * 5, num_rows * 5))

for i, region in enumerate(regions):
    # 选择当前region的子DataFrame
    sub_df = data_2015[data_2015['Region'] == region]
    
    # 创建该region的图
    G = nx.Graph()
    G.add_node(region)  # 添加地区节点
    for country in sub_df['Country']:
        G.add_node(country)  # 添加国家节点
        G.add_edge(region, country)  # 创建边
    
    # 计算布局
    pos = nx.circular_layout(G)  # 为了简单起见，我们使用圆形布局
    
    # 计算当前子图的行列位置
    row = i // num_cols
    col = i % num_cols
    
    # 在子图上绘制网络图
    nx.draw(G, pos, ax=axes[row, col], with_labels=True)
    axes[row, col].set_title(region)
    
    # 如果当前行的图表数量不足，关闭剩余的子图
    if i == len(regions) - 1:
        for j in range(i + 1, num_rows * num_cols):
            fig.delaxes(axes.flatten()[j])

plt.tight_layout()
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/1.png", format='png')
plt.show()

In [ ]:
#3. 分布图（Distribution Plots）
# 绘制Happiness Score的分布
plt.figure(figsize=(10, 6))
sns.histplot(combined_data['Happiness Score'], kde=True)
plt.title('Distribution of Happiness Scores')
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/2.png", format='png')
plt.show()


In [ ]:
# 对数据按照Happiness Rank进行排序
df_sorted =data_2015.sort_values('Happiness Rank')

# 绘制直方图
plt.figure(figsize=(20, 10))  # 可以根据国家数量调整大小
plt.bar(df_sorted['Country'], df_sorted['Happiness Rank'], color='skyblue')

# 添加标题和轴标签
plt.title('Happiness Rank by Country')
plt.xlabel('Country')
plt.ylabel('Happiness Rank')

# 旋转x轴标签，以便国家名可以清晰地显示
plt.xticks(rotation=90)

# 在显示之前调整布局
plt.tight_layout()

plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/3.png", format='png')
plt.show()


In [ ]:
# 首先对数据进行排序，这里假设按照'Economy (GDP per Capita)'排序
data_2015_sorted = data_2015.sort_values('Economy (GDP per Capita)')

# 创建一个新的图形
plt.figure(figsize=(14, 7))

# 对于每个属性，绘制一个线图
for feature in ['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)']:
    plt.plot(data_2015_sorted['Country'], data_2015_sorted[feature], marker='', label=feature)

# 添加图例
plt.legend()

# 设置标题和轴标签
plt.title('Attributes by Country')
plt.xlabel('Country')
plt.ylabel('Attribute Scores')

# 旋转x轴的标签，以便国家名可以清晰地显示
plt.xticks(fontsize=6, rotation=90)

# 使用tight_layout确保所有的标签和标题都适合
plt.tight_layout()
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/4.png", format='png')
# 显示图表
plt.show()

In [ ]:
# 绘制箱型图
plt.figure(figsize=(12, 8))  # 设置图形大小
boxplot = data_2015.boxplot(column=['Happiness Score'], by='Region', grid=False)

# 设置标题和轴标签
plt.title('Happiness Score by Region')
plt.suptitle('')  # 移除默认的子标题
plt.xlabel('Region')
plt.ylabel('Happiness Score')

# 旋转x轴的标签，以便区域名可以清晰地显示
plt.xticks(rotation=90)
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/5.png", format='png')
# 显示图表
plt.show()

In [ ]:
#5. 成对关系（Pair Plots）
# 选择数值类型的列来绘制成对关系
plt.figure(figsize=(12, 8))
sns.pairplot(combined_data[['Happiness Score', 'Economy (GDP per Capita)', 'Health (Life Expectancy)', 'Freedom']])
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/6.png", format='png')
plt.show()


In [ ]:
dataatt = pd.read_excel('P_Data_Extract_From_World_Development_Indicators (3).xlsx')


In [ ]:
dataatt 

In [ ]:

# 假设dataatt是你的DataFrame，并且已经加载
# 首先删除失业率列中的缺失值
data_clean = dataatt.dropna(subset=['Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]'])

# 计算每年的平均失业率
yearly_average = data_clean.groupby('Year')['Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]'].select(mean('column')).collect()[0][0]

# 绘制折线图
plt.figure(figsize=(12, 6))  # 调整图形大小
plt.plot(yearly_average.index, yearly_average.values, marker='o', linestyle='-', color='b')

# 添加标题和轴标签
plt.title('Average Unemployment Rate Over Years')
plt.xlabel('Year')
plt.ylabel('Average Unemployment Rate (%)')

# 显示网格
plt.grid(True)
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/7.png", format='png')
# 显示图表
plt.show()

In [ ]:
data_clean = dataatt.dropna(subset=['Political Stability and Absence of Violence/Terrorism: Estimate [PV.EST]'])

# 计算每年的平均政治稳定性评估
yearly_average = data_clean.groupby('Year')['Political Stability and Absence of Violence/Terrorism: Estimate [PV.EST]'].select(mean('column')).collect()[0][0]

# 绘制折线图
plt.figure(figsize=(12, 6))  # 调整图形大小
plt.plot(yearly_average.index, yearly_average.values, marker='o', linestyle='-', color='b', label='Average Political Stability')

# 填充折线图上方的颜色
#plt.fill_between(yearly_average.index, yearly_average.values, color='blue', alpha=0.1)

# 添加标题和轴标签
plt.title('Average Political Stability and Absence of Violence/Terrorism Estimate Over Years')
plt.xlabel('Year')
plt.ylabel('Average Estimate')

# 显示图例
plt.legend()

# 显示网格
plt.grid(True)
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/8.png", format='png')
# 显示图表
plt.show()


In [ ]:
data_clean = dataatt.dropna(subset=['Control of Corruption: Estimate [CC.EST]'])

# 计算每年的平均“控制腐败评估”
yearly_average = data_clean.groupby('Year')['Control of Corruption: Estimate [CC.EST]'].select(mean('column')).collect()[0][0]

# 绘制折线图
plt.figure(figsize=(12, 6))  # 调整图形大小
plt.plot(yearly_average.index, yearly_average.values, marker='o', linestyle='-', color='green', label='Average Control of Corruption')


# 添加标题和轴标签
plt.title('Average Control of Corruption Estimate Over Years')
plt.xlabel('Year')
plt.ylabel('Average Estimate')

# 显示图例
plt.legend()

# 显示网格
plt.grid(True)
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/9.png", format='png')
# 显示图表
plt.show()

In [ ]:
data_clean = dataatt.dropna(subset=[
    'Merchandise exports (current US$) [TX.VAL.MRCH.CD.WT]',
    'Merchandise imports (current US$) [TM.VAL.MRCH.CD.WT]',
    'Merchandise trade (% of GDP) [TG.VAL.TOTL.GD.ZS]'
])

# 计算每年的平均值
yearly_average_exports = data_clean.groupby('Country')['Merchandise exports (current US$) [TX.VAL.MRCH.CD.WT]'].select(mean('column')).collect()[0][0]
yearly_average_imports = data_clean.groupby('Country')['Merchandise imports (current US$) [TM.VAL.MRCH.CD.WT]'].select(mean('column')).collect()[0][0]
yearly_average_trade = data_clean.groupby('Country')['Merchandise trade (% of GDP) [TG.VAL.TOTL.GD.ZS]'].select(mean('column')).collect()[0][0]

# 绘制折线图
plt.figure(figsize=(14, 8))  # 调整图形大小
plt.plot(yearly_average_exports.index, yearly_average_exports.values,  linestyle='-', color='blue', label='Merchandise Exports (US$)')
plt.plot(yearly_average_imports.index, yearly_average_imports.values,  linestyle='-', color='red', label='Merchandise Imports (US$)')
plt.plot(yearly_average_trade.index, yearly_average_trade.values,  linestyle='-', color='green', label='Merchandise Trade (% of GDP)')

# 添加标题和轴标签
plt.title('Merchandise Trade Metrics Over Years')
plt.xlabel('Country')
plt.ylabel('Values')
plt.xticks(fontsize=4, rotation=90)
# 显示图例
plt.legend()

# 显示网格
plt.grid(True)
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/10.png", format='png')
# 显示图表
plt.show()

In [ ]:
combined_data

In [ ]:
data_clean = combined_data.dropna(subset=['Happiness Score'])

# 计算每年的平均Happiness Score
yearly_average_happiness = data_clean.groupby('Year')['Happiness Score'].select(mean('column')).collect()[0][0]

# 绘制折线图
plt.figure(figsize=(12, 6))  # 调整图形大小
plt.plot(yearly_average_happiness.index, yearly_average_happiness.values, marker='o', linestyle='-', color='teal')

# 添加标题和轴标签
plt.title('Average Happiness Score Over Years')
plt.xlabel('Year')
plt.ylabel('Average Happiness Score')

# 显示网格
plt.grid(True)
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/11.png", format='png')
# 显示图表
plt.show()

In [ ]:
data_2015

In [ ]:
# 清洁数据
data_2015_clean = data_2015.dropna(subset=['Economy (GDP per Capita)', 'Happiness Score'])

# 绘制二维直方图
plt.figure(figsize=(10, 6))
plt.hist2d(data_2015_clean['Economy (GDP per Capita)'], data_2015_clean['Happiness Score'], bins=30, cmap='Blues')

# 添加颜色条
plt.colorbar(label='Count in bin')

# 添加标题和轴标签
plt.title('2D Histogram of Happiness Score vs. Economy (GDP per Capita)')
plt.xlabel('Economy (GDP per Capita)')
plt.ylabel('Happiness Score')
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/12.png", format='png')
# 显示图表
plt.show()

In [ ]:
data_2015_clean = data_2015.dropna(subset=['Freedom', 'Happiness Score'])

# 创建散点图
plt.figure(figsize=(10, 6))
plt.scatter(data_2015_clean['Happiness Score'], data_2015_clean['Freedom'], alpha=0.5, color='lightblue')

# 设置背景颜色为黑色
plt.gca().set_facecolor('black')

# 添加标题和轴标签
plt.title('Happiness Score vs. Freedom')
plt.xlabel('Happiness Score')
plt.ylabel('Freedom')

# 设置轴标签和标题的颜色为白色
plt.tick_params(colors='white')
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/13.png", format='png')
# 显示图表
plt.show()

In [ ]:
#1. 散点图（Scatter Plots）
# 设置绘图风格
sns.set(style="whitegrid")

# 绘制Economy和Happiness Score的关系
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Economy (GDP per Capita)', y='Happiness Score', data=combined_data)
plt.title('Economy vs Happiness Score')
plt.show()

In [ ]:
axes = pd.plotting.scatter_matrix(combined_data, alpha=0.2, figsize=(12, 12), diagonal='kde', marker='o', grid=True)

# 调整子图之间的间隔
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, wspace=0.4, hspace=0.4)

plt.show()

In [ ]:
#2. 相关矩阵（Correlation Matrix）
plt.figure(figsize=(10, 8))
data1=combined_data.drop(columns=['Country'])
correlation_matrix = data1.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Matrix of Variables')
plt.show()


In [ ]:
#4. 箱形图（Box Plots）
# 绘制各变量的箱形图，比较不同因素
plt.figure(figsize=(12, 8))
sns.boxplot(data=combined_data[['Freedom', 'Generosity', 'Trust (Government Corruption)']])
plt.title('Box Plot of Various Factors')
plt.show()


In [ ]:
#先讲属性数据由于缺失数据表示成。。而全是字符串格式

In [ ]:
data_attribute.columns = [col.split(' (')[0] for col in data_attribute.columns]

# 计算每列的缺失值数量
missing_values = data_attribute.isnull().sum()

# 循环遍历每列的缺失值数量并打印
for column, missing_count in missing_values.items():
    print(f"{column}: {missing_count} ")



In [ ]:
# 计算每列的缺失值数量
missing_values = combined_data.isnull().sum()

# 循环遍历每列的缺失值数量并打印
for column, missing_count in missing_values.items():
    print(f"{column}: {missing_count} ")


In [ ]:
print(data_attribute.dtypes)

In [ ]:
duplicates = data_attribute[data_attribute.duplicated()]
print("Number of duplicate entries:", duplicates.shape[0])

In [ ]:
duplicates = combined_data[combined_data.duplicated()]
print("Number of duplicate entries:", duplicates.shape[0])

In [ ]:
# 计算每列的缺失值比例
missing_ratio = data_attribute.isnull().select(mean('column')).collect()[0][0] * 100

# 对缺失率进行排序，从高到低
missing_ratio_sorted = missing_ratio.sort_values(ascending=False)

# 打印每列的缺失率，按从高到低排序
for column, missing_count in missing_ratio_sorted.items():
    print(f"{column}: {missing_count} ")

In [ ]:
# 计算每列的缺失值比例
missing_ratio2 = combined_data.isnull().select(mean('column')).collect()[0][0] * 100

# 对缺失率进行排序，从高到低
missing_ratio_sorted2 = missing_ratio2.sort_values(ascending=False)

# 打印每列的缺失率，按从高到低排序
for column, missing_count in missing_ratio_sorted2.items():
    print(f"{column}: {missing_count} ")

In [ ]:
# 定义阈值
threshold = 30

# 找出缺失率高于40%的列
columns_to_drop = missing_ratio[missing_ratio > threshold].index

# 删除这些列
data_attribute.drop(columns=columns_to_drop, inplace=True)




In [ ]:
# 计算每列的缺失值比例
missing_ratio = data_attribute.isnull().select(mean('column')).collect()[0][0] * 100

# 对缺失率进行排序，从高到低
missing_ratio_sorted = missing_ratio.sort_values(ascending=False)

# 打印每列的缺失率，按从高到低排序
for column, missing_count in missing_ratio_sorted.items():
    print(f"{column}: {missing_count} ")

In [ ]:
duplicates = data_attribute[data_attribute.duplicated()]
print("Number of duplicate entries:", duplicates.shape[0])

In [ ]:
data_attribute = data_attribute.drop_duplicates()
duplicates = data_attribute[data_attribute.duplicated()]
print("Number of duplicate entries:", duplicates.shape[0])

In [ ]:
data_2015

In [ ]:
data_2015_clean = data_2020.dropna(subset=['Trust (Government Corruption)'])

# 绘制箱型图
plt.figure(figsize=(12, 7))
data_2015_clean.boxplot(column='Trust (Government Corruption)', by='Region', grid=False)

# 添加图表标题和轴标签
plt.title('Trust (Government Corruption) by Region')
plt.suptitle('')  # 去除默认的副标题
plt.xlabel('Region')
plt.ylabel('Trust (Government Corruption)')

# 优化x轴标签的显示
plt.xticks(rotation=90, ha='right')
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/21.png", format='png')
# 显示图表
plt.show()

In [ ]:
# 清除包含缺失值的行
data_clean = dataatt.dropna(subset=['Unemployment with advanced education (% of total labor force with advanced education) [SL.UEM.ADVN.ZS]'])

# 绘制二维直方图
plt.figure(figsize=(10, 5))
plt.hist(data_clean['Unemployment with advanced education (% of total labor force with advanced education) [SL.UEM.ADVN.ZS]'], bins=50, color='navy')

# 添加标题和轴标签
plt.title('Unemployment rate after higher education')
plt.xlabel('Unemployment with advanced education (% of total labor force with advanced education) [SL.UEM.ADVN.ZS]')
plt.ylabel('Frequency')

# 设置图形背景颜色为白色
plt.gca().set_facecolor('white')

# 显示网格
plt.grid(True)
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/22.png", format='png')
# 显示图表
plt.show()

In [ ]:
# 选择数据类型为数值的列
numeric_cols = data_attribute.select_dtypes(include=['int64', 'float64'])
# 设定每行显示的图形数
n_cols = 3
n_rows = (len(numeric_cols.columns) + n_cols - 1) // n_cols  # 确保有足够的行来展示所有图形

# 创建一个足够大的画布来容纳所有的直方图
plt.figure(figsize=(n_cols * 5, n_rows * 4))  # 宽度为5*n_cols, 高度为4*n_rows

for i, column in enumerate(numeric_cols.columns, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.histplot(data_attribute[column], kde=True, color='skyblue')
    plt.title(f'Distribution of {column}')
    plt.xlabel('')
    plt.ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
#combined_data = pd.concat([data_2013, data_2015, data_2016, data_2017, data_2018, 
                           data_2019, data_2020, data_2021, data_2022, data_2023], 
                          axis=0, ignore_index=True)

print("Shape of the 2013 year dataset:", data_2013.shape)
print("Shape of the combined dataset:", combined_data.shape)

In [ ]:
data_attribute.replace('..', np.nan, inplace=True)

combined_data['Country'] = combined_data['Country'].str.replace('*', '', regex=False)

#问题二： 将列名'Time'改为'Year'，两个数据集不同意
data_attribute.rename(columns={'Time': 'Year'}, inplace=True)
 

In [ ]:
numeric_df = data_attribute.select_dtypes(include=['int64', 'float64'])
imputer = IterativeImputer(max_iter=10, random_state=0)
numeric_df_filled = pd.DataFrame(imputer.fit_transform(numeric_df), columns=numeric_df.columns)

data_attribute.update(numeric_df_filled)

data_attribute



In [ ]:
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import KNNImputer, IterativeImputer

#numeric_df = data_attribute.select_dtypes(include=['int64', 'float64'])

# 创建KNN填充器实例
#knn_imputer = KNNImputer(n_neighbors=5)

# 应用填充器
#numeric_df_filled_knn = pd.DataFrame(knn_imputer.fit_transform(numeric_df), columns=numeric_df.columns)

# 检查填充后的结果
#print(numeric_df_filled_knn)

# 合并填充后的数值数据与原始非数值数据
#data_attribute.update(numeric_df_filled_knn)

# 查看完整的DataFrame结果
#print(data_attribute)

In [ ]:
country_name_mapping = {
    'Congo (Brazzaville)': 'Congo, Rep.',
    'Czech Republic': 'Czechia',
    'Egypt': 'Egypt, Arab Rep.',
    'Gambia': 'Gambia, The',
    'Hong Kong S.A.R. of China': 'Hong Kong SAR, China',
    'Iran': 'Iran, Islamic Rep.',
    'Kyrgyzstan': 'Kyrgyz Republic',
    'Laos': 'Lao PDR',
  #  'North Cyprus': '
   # 'Palestinian Territories': '
    'Russia': 'Russian Federation',
    'Slovakia': 'Slovak Republic',
    'South Korea': 'Korea, Rep.',
    #'Swaziland': '
    #'Taiwan Province of China': '
    'Turkey': 'Turkiye',
    'Venezuela': 'Venezuela, RB',
    'Vietnam': 'Viet Nam',
    'Yemen': 'Yemen, Rep.',
    'Zimbabwe': 'Zimbabwe'

}

In [ ]:
data_attribute['Country'] = data_attribute['Country'].map(country_name_mapping).fillna(data_attribute['Country'])
combined_data['Country'] = combined_data['Country'].map(country_name_mapping).fillna(combined_data['Country'])


In [ ]:
# 筛选数值型数据
numeric_data = data_attribute.select_dtypes(include=['float64', 'int64'])

# 绘图设置
num_columns = numeric_data.shape[1]
num_rows = (num_columns + 3) // 4  # 计算需要的行数
plt.figure(figsize=(16, 4 * num_rows))  # 设置画布大小

# 绘制箱型图
for i, column in enumerate(numeric_data.columns, 1):
    plt.subplot(num_rows, 4, i)
    sns.boxplot(y=numeric_data[column])
    plt.title(column)

plt.tight_layout()
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/20.png", format='png')
plt.show()

In [ ]:
#data5在后面定义的
numeric_data =data_attribute.select_dtypes(include=['float64', 'int64'])
def outliers_iqr(data):
    Q1 = data.quantile(0.05)
    Q3 = data.quantile(0.95)
    IQR = Q3 - Q1
    outliers_mask = (data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))
    return outliers_mask

print("\nOutliers detected by IQR Method:")
for column in numeric_data.columns:
    mask = outliers_iqr(numeric_data[column])
    print(f"{column} ： {mask.sum()} ")

In [ ]:
def replace_with_nearest(data, mask):
    import numpy as np
    normal_values = data[~mask]  # 非异常值
    for idx in data[mask].index:
        closest_idx = np.abs(normal_values - data.loc[idx]).idxmin()
        closest_value = data.loc[closest_idx]
        data.at[idx] = closest_value
    return data



numeric_data = data_attribute.select_dtypes(include=['float64', 'int64'])


def outliers_iqr(data):
    Q1 = data.quantile(0.05)
    Q3 = data.quantile(0.95)
    IQR = Q3 - Q1
    outliers_mask = (data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))
    return outliers_mask


for column in numeric_data.columns:
    mask = outliers_iqr(numeric_data[column])
    numeric_data[column] = replace_with_nearest(numeric_data[column], mask)

print("\nAfter processing outliers:")
print(numeric_data)
data5.update(numeric_data)


In [ ]:
data_2013['Year'] = 2013
data_2015['Year'] = 2015
data_2016['Year'] = 2016
data_2017['Year'] = 2017
data_2018['Year'] = 2018
data_2019['Year'] = 2019
data_2020['Year'] = 2020
data_2021['Year'] = 2021
data_2022['Year'] = 2022
data_2023['Year'] = 2023

In [ ]:
data_attribute['COVID_Impact'] = data_attribute['Year'].apply(lambda x: 1 if x in [2019,2020, 2021] else 0)
data_attribute

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(data_attribute['COVID_Impact'], bins=[-0.5, 0.5, 1.5], color='blue', edgecolor='black')
plt.title('Histogram of COVID Impact')
plt.xlabel('COVID Impact Score')
plt.ylabel('Frequency')
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/32.png", format='png')
plt.show()

In [ ]:
# 根据Year和Country列合并两个数据集
# 这里使用left join，意味着以combined_data的行为基准
data3 = pd.merge(combined_data, data_attribute, on=['Year', 'Country'], how='left')

data3

In [ ]:
#data3['Happiness Ranking'] = data3['Happiness Ranking'].astype(int)

In [ ]:
# 计算每列的缺失值数量
missing_values = data3.isnull().sum()

# 循环遍历每列的缺失值数量并打印
for column, missing_count in missing_values.items():
    print(f"{column}: {missing_count} ")


In [ ]:
#data3[data3.isnull().any(axis=1)]
missing_count = data3.isnull().sum(axis=1)
#data3 = data3[missing_count <= 3]
data3[data3.isnull().any(axis=1)]
numeric_data = data3.select_dtypes(include=[np.number])
imputer = IterativeImputer(max_iter=10, random_state=0)
numeric_data_imputed = pd.DataFrame(imputer.fit_transform(numeric_data), columns=numeric_data.columns)
data3.update(numeric_data_imputed)



In [ ]:
data3

In [ ]:
#2. 相关矩阵（Correlation Matrix）
plt.figure(figsize=(10, 8))
data1=combined_data.drop(columns=['Country'])
correlation_matrix = data1.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Matrix of Variables')
plt.show()


In [ ]:
# 选择数值型数据
numeric_data = data3.select_dtypes(include=[np.number])  # 包括int和float

# 计算相关系数矩阵
correlation_matrix = numeric_data.corr()

# 绘制热图
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=False, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Matrix of Numeric Features")
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/39.png", format='png')
plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
data4=data3
data3.drop('Happiness Rank', axis=1, inplace=True)
numeric_data = data3.select_dtypes(include=[np.number])  # 包括int和float
# 假设data3是你的DataFrame，'Happiness Score'是目标变量
X = numeric_data.drop('Happiness Score', axis=1)  # 删除目标变量列
y = data3['Happiness Score']

# 选择最佳的K个特征
selector = SelectKBest(score_func=f_regression, k=15)  # 选择10个最重要的特征
X_new = selector.fit_transform(X, y)

# 查看所选特征
selected_features = pd.DataFrame(selector.inverse_transform(X_new), 
                                 index=data3.index, 
                                 columns=X.columns)
selected_columns = selected_features.columns[selector.get_support()]
print("Selected features based on f_regression:", selected_columns)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X, y)

feature_importances = pd.Series(model.feature_importances_, index=X.columns)
selected_features = feature_importances.nlargest(25) 
print("Most important features based on RandomForest:\n", selected_features)


In [ ]:
selected_features = feature_importances.nlargest(20)
X = X[selected_features.index]
non_numeric_data = data3.select_dtypes(exclude=[np.number])
data5 = pd.concat([X, y, non_numeric_data], axis=1)

In [ ]:
data5

In [ ]:
# Define a dictionary with old column names as keys and new column names as values
new_column_names = {
    'Fixed broadband subscriptions': 'Fixed_Broadband',
    'Fixed telephone subscriptions': 'Fixed_Telephone',
    'Economy (GDP per Capita)': 'GDP_per_Capita',
    'Employment to population ratio, ages 15-24, total': 'Youth_Emp_Ratio',
    'Health (Life Expectancy)': 'Health_Life_Exp',
    'Adjusted savings: carbon dioxide damage': 'CO2_Damage',
    'Freedom': 'Freedom',
    'Individuals using the Internet': 'Internet_Users',
    'Life expectancy at birth, total': 'Life_Expectancy',
    'Labor force, female': 'Female_Labor_Force',
    'Trust (Government Corruption)': 'Gov_Trust',
    'Mortality rate, neonatal': 'Neonatal_Mortality',
    'Access to electricity': 'Electricity_Access',
    'Agriculture, forestry, and fishing, value added': 'Agri_Value_Added',
    'Computer, communications and other services': 'Tech_Services',
    'Merchandise exports': 'Merch_Exports',
    'GDP': 'GDP_Total',
    'Insurance and financial services': 'Insur_Fin_Services',
    'Adolescent fertility rate': 'Adol_Fertility',
    'Labor force, total [SL.TLF.TOTL.IN]': 'Total_Labor_Force',
    'Happiness Score': 'Happiness Score',
    'Country': 'Country',
    'Year': 'Year',
    'Time Code': 'Time_Code',
    'Country Code': 'Country_Code'
}

data5.rename(columns=new_column_names, inplace=True)
data5

In [ ]:
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
numeric_data = data5.select_dtypes(include=[np.number])  # 包括int和float
pt = PowerTransformer(method='yeo-johnson')
data_transformed = pt.fit_transform(numeric_data.values.reshape(-1, 1))


In [ ]:
#stander
scaler = StandardScaler()
data_transformed= scaler.fit_transform(data_transformed)
data5.update(data_transformed)

In [ ]:
data5_new = data5.drop(['GDP_per_Capita', 'Health_Life_Exp', 'GDP_Total'], axis=1)
#, 'Total_Labor_Force', 'Agri_Value_Added'

# 选择数据类型为数值的列
numeric_cols = data5_new.select_dtypes(include=['int64', 'float64'])
# 设定每行显示的图形数
n_cols = 4
n_rows = (len(numeric_cols.columns) + n_cols - 1) // n_cols  # 确保有足够的行来展示所有图形

# 创建一个足够大的画布来容纳所有的直方图
plt.figure(figsize=(n_cols * 5, n_rows * 4))  # 宽度为5*n_cols, 高度为4*n_rows

for i, column in enumerate(numeric_cols.columns, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.histplot(data5_new[column], kde=True, color='skyblue')
    plt.title(f'Distribution of {column}')
    plt.xlabel('')
    plt.ylabel('')

plt.tight_layout()
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/44.png", format='png')
plt.show()

In [ ]:
data5['COVID_Impact'] = data5['Year'].apply(lambda x: 1 if x in [2020, 2021] else 0)
covid_impact_dummies = pd.get_dummies(data5['COVID_Impact'], prefix='COVID_Impact')
data5 = pd.concat([data5, covid_impact_dummies], axis=1)
data5 = data5.drop('COVID_Impact', axis=1)

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
data_transformed = scaler.fit_transform(numeric_data)
numeric_data.update(data_transformed)
#numeric_data = data5.select_dtypes(include=[np.number]) 
X = numeric_data.drop('Happiness Score', axis=1)
X = pd.concat([X, covid_impact_dummies], axis=1)
y = data5['Happiness Score']


In [ ]:
X

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

alpha_range = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]

lasso = Lasso()
lasso_cv = GridSearchCV(estimator=lasso, param_grid={'alpha': alpha_range}, scoring='neg_mean_squared_error', cv=5)
lasso_cv.fit(X, y)
print(f"Best alpha: {lasso_cv.best_params_['alpha']}")
print(f"MSE: {lasso_cv.best_score_}")
model = lasso_cv.best_estimator_



In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


wcss = []
for i in range(1, 11):  # try1 to10 cluster
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 8))
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/46.png", format='png')
plt.show()


In [ ]:
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


# 使用PolynomialFeatures仅对训练数据进行转换
interaction = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
features = X_train.columns
X_train_interactions = interaction.fit_transform(X_train)
interaction_features = interaction.get_feature_names_out(features)
X_train_interactions_df = pd.DataFrame(X_train_interactions, columns=interaction_features)

model3 = LinearRegression()




In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
numeric_data = data5.select_dtypes(include=[np.number]) 
#X = numeric_data.drop('Happiness Score', axis=1)
#y = data5['Happiness Score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test.shape[0]

In [ ]:
X_train

In [ ]:
model.fit(X_train, y_train)

In [ ]:
kmeans.fit(X)

In [ ]:
model3.fit(X_train_interactions_df, y_train)

In [ ]:
import statsmodels.api as sm
X1 = sm.add_constant(X)  # 添加截距项
# 假设 y 是你的目标变量，X 是包含截距项的特征矩阵
model_1 = sm.OLS(y, X1).fit()
# 打印出完整的统计摘要
model_1.summary()


In [ ]:
# 获取模型系数和截距
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficients'])
intercept = model.intercept_

coefficients_sorted = coefficients.sort_values(by='Coefficients', ascending=False)

# 打印排序后的系数和截距
print("Coefficients:\n", coefficients_sorted)
print("Intercept:", intercept)


In [ ]:
clusters = kmeans.fit_predict(X)
# 将聚类结果添加到未标准化的原始数据集中
numeric_data['Cluster'] = clusters
cluster_means = numeric_data.groupby('Cluster').select(mean('column')).collect()[0][0]
cluster_means.T



In [ ]:
# 重置索引，确保数据对齐
X_train_interactions_df = X_train_interactions_df.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# 现在重新添加常数项和拟合模型
X_train_interactions_sm = sm.add_constant(X_train_interactions_df)
model_sm = sm.OLS(y_train, X_train_interactions_sm).fit()

# 创建一个 DataFrame 来存储系数、p 值
results_df = pd.DataFrame({
    'Coefficients': model_sm.params,
    'P-values': model_sm.pvalues
})
results_df = results_df.sort_values(by='P-values', ascending=True)
# 过滤出包含 "COVID_Impact" 的系数
covid_impact_df = results_df[results_df.index.str.contains('COVID_Impact')]
covid_impact_df


In [ ]:
X_test_interactions = interaction.transform(X_test)
X_test_interactions_df = pd.DataFrame(X_test_interactions, columns=interaction_features)
y_pred = model3.predict(X_test_interactions_df)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

In [ ]:
# 预测值与实际值的对比图
y_pred = model.predict(X_test)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.75, color='b')  # 预测值和实际值的散点图
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)  # 添加一条完美预测的对角线
plt.show()

# 各特征系数的影响力
coefficients.plot(kind='barh')
plt.title('Feature Coefficients')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)


In [ ]:
# 特征列列表
features = X.columns

# 设置画布大小
plt.figure(figsize=(20, 15))

# 遍历特征列表，为每个特征绘图
for i, feature in enumerate(features, 1):
    plt.subplot((len(features) + 4) // 4, 4, i)  # 确保足够的子图空间
    sns.scatterplot(x=data5['Happiness_Score'], y=data5[feature])
    plt.title(f'{feature}')
    plt.ylabel('')
    plt.xlabel('Happiness Score')
# 调整子图间距
plt.tight_layout()
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/60.png", format='png')
plt.show()


In [ ]:
data5.columns 

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
data_pca = pca.fit_transform(numeric_data)

# 将PCA结果和聚类标签组合
data_pca_clustered = pd.DataFrame(data_pca, columns=['PC1', 'PC2'])
data_pca_clustered['Cluster'] = clusters

# 可视化
plt.figure(figsize=(10, 8))
sns.scatterplot(data=data_pca_clustered, x='PC1', y='PC2', hue='Cluster', palette='viridis', alpha=0.7)
plt.title('Clusters visualized with PCA')
plt.show()


In [ ]:
numeric_data['Cluster'] = clusters
# 计算每个簇的大小
cluster_sizes = numeric_data['Cluster'].value_counts()

# 定义颜色数组，数量应与簇的数量一致
colors = plt.cm.tab20.colors[:len(cluster_sizes)]

# 绘制饼图
plt.figure(figsize=(8, 8))  # 设置图形的大小
plt.pie(cluster_sizes, labels=cluster_sizes.index, autopct='%1.1f%%', colors=colors, startangle=90)
plt.title('Cluster Size Distribution')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/61.png", format='png')
# 显示图形
plt.show()


In [ ]:
# 确保Seaborn的样式
sns.set(style="whitegrid")

# 创建一个箱型图，表示不同簇的Happiness Score
plt.figure(figsize=(10, 6))  # 设置图形的大小
sns.boxplot(x='Cluster', y='Happiness Score', data=numeric_data)

plt.title('Happiness Score Distribution by Cluster')
plt.xlabel('Cluster')
plt.ylabel('Happiness Score')
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/62.png", format='png')
# 显示图形
plt.show()

In [ ]:
cluster_sizes = pd.Series({
    0: 895,
    1: 377,
    2: 223
})

In [ ]:
# 预测值与实际值的对比图

y_pred = model3.predict(X_test_interactions_df)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.75, color='b')  # 预测值和实际值的散点图
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)  # 添加一条完美预测的对角线
plt.show()

# 各特征系数的影响力
coefficients.plot(kind='barh')
plt.title('Feature Coefficients')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)


In [ ]:
sns.set(style="whitegrid")

# 创建一个箱型图，表示COVID_Impact为0和1时Happiness Score的分布
plt.figure(figsize=(8, 6))  # 设置图形的大小
sns.boxplot(x='COVID_Impact', y='Happiness_Score', data=numeric_data)

plt.title('Distribution of Happiness Score by COVID Impact')
plt.xlabel('COVID Impact')
plt.ylabel('Happiness_Score')
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/63.png", format='png')
# 显示图形
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 设置Seaborn样式
sns.set(style="whitegrid")

# 准备数据：分别筛选COVID_Impact为0和1的数据，并计算每个Happiness Score的Life Expectancy平均值
mean_life_exp_covid_0 = numeric_data[numeric_data['COVID_Impact'] == 0].groupby('Happiness_Score')['Life_Expectancy'].select(mean('column')).collect()[0][0].reset_index()
mean_life_exp_covid_1 = numeric_data[numeric_data['COVID_Impact'] == 1].groupby('Happiness_Score')['Life_Expectancy'].select(mean('column')).collect()[0][0].reset_index()

# 创建一个1x2的子图布局
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

# 在第一个子图中绘制COVID_Impact为0的数据
sns.lineplot(x='Happiness_Score', y='Life_Expectancy', data=mean_life_exp_covid_0, ax=ax[0], marker='o')
ax[0].set_title('Life Expectancy vs. Happiness Score (COVID Impact = 0)')
ax[0].set_xlabel('Happiness Score')
ax[0].set_ylabel('Life Expectancy')

# 在第二个子图中绘制COVID_Impact为1的数据
sns.lineplot(x='Happiness_Score', y='Life_Expectancy', data=mean_life_exp_covid_1, ax=ax[1], marker='o')
ax[1].set_title('Life Expectancy vs. Happiness Score (COVID Impact = 1)')
ax[1].set_xlabel('Happiness Score')
ax[1].set_ylabel('Life Expectancy')

# 调整子图间距
plt.tight_layout()
plt.savefig("/Users/zhoudaozhen/Desktop/722/iteration3/Figures/65.png", format='png')
# 显示图形
plt.show()


In [ ]:
# 基本统计评估
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

# 分析模型的适用性和限制
print("\nModel Evaluation:")
print("The model explains {:.2f}% of the total variance in the dependent variable.".format(r2 * 100))
print("MSE indicates the average squared difference between actual and predicted values:", mse)

# 残差分析
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.residplot(x=y_pred, y=residuals, lowess=True, color="g")
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Values')
plt.show()

# 检查残差的正态分布
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(numeric_data, clusters)
print(f"The average silhouette_score is : {silhouette_avg}")
